In [2]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam

# Input shape
inputs = Input(shape=(128, 128, 3))

# Load EfficientNetV2B0 base model without top layers
base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_tensor=inputs)

# Freeze the first 100 layers
for layer in base_model.layers[:100]:
    layer.trainable = False

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(10, activation='softmax')(x)

# Final model
model = Model(inputs, outputs)

# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 128, 128,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 128, 128,  │          0 │ rescaling_1[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 64, 64,    │        864 │ normalization_1[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 64, 64,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 64, 64,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 64, 64,    │      4,608 │ stem_activation[… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 64, 64,    │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 64, 64,    │          0 │ block1a_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 32, 32,    │      9,216 │ block1a_project_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 32, 32,    │        256 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 32, 32,    │          0 │ block2a_expand_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_co… │ (None, 32, 32,    │      2,048 │ block2a_expand_a… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_bn  │ (None, 32, 32,    │        128 │ block2a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_conv │ (None, 32, 32,    │     36,864 │ block2a_project_… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_bn   │ (None, 32, 32,    │        512 │ block2b_expand_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_act… │ (None, 32, 32,    │          0 │ block2b_expand_b

 Total params: 5,932,122 (22.63 MB)

 Trainable params: 5,168,458 (19.72 MB)

 Non-trainable params: 763,664 (2.91 MB)

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# Paths
data_path = "modified-dataset"
IMG_SIZE = (128, 128)
BATCH_SIZE = 100
AUTOTUNE = tf.data.AUTOTUNE

# Load datasets again
train_ds = image_dataset_from_directory(
    f"{data_path}/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

val_ds = image_dataset_from_directory(
    f"{data_path}/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

test_ds = image_dataset_from_directory(
    f"{data_path}/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

# Data augmentation and preprocessing
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

def prepare(ds, augment=False):
    ds = ds.map(lambda x, y: (preprocess_input(x), y))
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    return ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

# Final datasets
train_ds = prepare(train_ds, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)


Found 2400 files belonging to 10 classes.
Found 300 files belonging to 10 classes.
Found 300 files belonging to 10 classes.


In [12]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stop]
)


Epoch 1/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 316s 7s/step - accuracy: 0.1991 - loss: 2.1748 - val_accuracy: 0.5433 - val_loss: 1.6520
Epoch 2/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 154s 6s/step - accuracy: 0.6630 - loss: 1.4216 - val_accuracy: 0.7700 - val_loss: 1.0689
Epoch 3/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 195s 6s/step - accuracy: 0.8384 - loss: 0.8854 - val_accuracy: 0.8600 - val_loss: 0.7012
Epoch 4/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 153s 6s/step - accuracy: 0.9091 - loss: 0.5412 - val_accuracy: 0.8900 - val_loss: 0.5072
Epoch 5/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 145s 6s/step - accuracy: 0.9413 - loss: 0.3732 - val_accuracy: 0.9033 - val_loss: 0.4002
Epoch 6/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 209s 6s/step - accuracy: 0.9588 - loss: 0.2653 - val_accuracy: 0.9200 - val_loss: 0.3387
Epoch 7/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 197s 6s/step - accuracy: 0.9724 - loss: 0.1836 - val_accuracy: 0.9367 - val_loss: 0.2888
Epoch 8/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 146s 6s/step - accuracy: 0.9850 - loss: 0.1270 - val_accuracy: 0.9400 - v

In [17]:
# Save the trained model to a local folder
model.save("e_waste_classifier_model.h5")

print("✅ Model saved as 'e_waste_classifier_model.h5'")


✅ Model saved as 'e_waste_classifier_model.h5'


In [19]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"\n📈 Test Accuracy: {test_accuracy:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9517 - loss: 0.1687

📈 Test Accuracy: 0.9433
